In [1]:
import pandas as pd

ts_data = pd.read_parquet('../data/transformed/ts_data_2023_01.parquet')

ts_data

,pickup_hour,rides,pickup_location_id
0,2023-01-01 00:00:00,19,4
1,2023-01-01 01:00:00,28,4
2,2023-01-01 02:00:00,43,4
3,2023-01-01 03:00:00,33,4
4,2023-01-01 04:00:00,12,4
...,...,...,...
191203,2023-01-31 19:00:00,0,2
191204,2023-01-31 20:00:00,0,2
191205,2023-01-31 21:00:00,0,2
191206,2023-01-31 22:00:00,0,2


In [4]:
ts_data_one_location = ts_data.loc[ts_data.pickup_location_id == 43, :].reset_index(drop=True)
ts_data_one_location

,pickup_hour,rides,pickup_location_id
0,2023-01-01 00:00:00,93,43
1,2023-01-01 01:00:00,81,43
2,2023-01-01 02:00:00,30,43
3,2023-01-01 03:00:00,15,43
4,2023-01-01 04:00:00,4,43
...,...,...,...
739,2023-01-31 19:00:00,81,43
740,2023-01-31 20:00:00,49,43
741,2023-01-31 21:00:00,44,43
742,2023-01-31 22:00:00,35,43


In [8]:
def get_cutoff_indices(
        data: pd.DataFrame,
        n_features: int,
        step_size: int
) -> list:
    
    stop_position = len(data) - 1

    subseq_first_idx = 0
    subseq_mid_idx = n_features
    subseq_last_idx = n_features + 1
    indices = []

    while subseq_last_idx <= stop_position:
        indices.append((subseq_first_idx, subseq_mid_idx, subseq_last_idx))

        subseq_first_idx += step_size
        subseq_mid_idx += step_size
        subseq_last_idx += step_size

    return indices

In [9]:
n_features = 24
step_size = 1

indices = get_cutoff_indices(
    ts_data_one_location,
    n_features,
    step_size
)
indices[:5]

[(0, 24, 25), (1, 25, 26), (2, 26, 27), (3, 27, 28), (4, 28, 29)]

In [10]:
len(ts_data_one_location)

744

In [14]:
import numpy as np

n_examples = len(indices)
x = np.ndarray(shape=(n_examples, n_features), dtype=np.float32)
y = np.ndarray(shape=(n_examples), dtype=np.float32)
pickup_hours = []

for i, idx in enumerate(indices):
    x[i, :] = ts_data_one_location.iloc[idx[0]:idx[1]]['rides'].values
    y[i] = ts_data_one_location.iloc[idx[1]:idx[2]]['rides'].values[0]
    pickup_hours.append(ts_data_one_location.iloc[idx[1]]['pickup_hour'])


In [15]:
print(f'{x.shape=}')
print(f'{x=}')
print(f'{pickup_hours[:5]=}')

x.shape=(719, 24)
x=array([[ 93.,  81.,  30., ...,  41.,  18.,  13.],
       [ 81.,  30.,  15., ...,  18.,  13.,   2.],
       [ 30.,  15.,   4., ...,  13.,   2.,   0.],
       ...,
       [ 50.,  33.,  16., ..., 108.,  88.,  81.],
       [ 33.,  16.,  17., ...,  88.,  81.,  49.],
       [ 16.,  17.,   3., ...,  81.,  49.,  44.]], dtype=float32)
pickup_hours[:5]=[Timestamp('2023-01-02 00:00:00'), Timestamp('2023-01-02 01:00:00'), Timestamp('2023-01-02 02:00:00'), Timestamp('2023-01-02 03:00:00'), Timestamp('2023-01-02 04:00:00')]


In [16]:
features_one_location = pd.DataFrame(
    x,
    columns=[f'rides_previous_{i+1}_hour' for i in reversed(range(n_features))]
)
features_one_location

,rides_previous_24_hour,rides_previous_23_hour,rides_previous_22_hour,rides_previous_21_hour,rides_previous_20_hour,rides_previous_19_hour,rides_previous_18_hour,rides_previous_17_hour,rides_previous_16_hour,rides_previous_15_hour,...,rides_previous_10_hour,rides_previous_9_hour,rides_previous_8_hour,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour
0,93.0,81.0,30.0,15.0,4.0,4.0,4.0,12.0,12.0,23.0,...,106.0,120.0,104.0,65.0,39.0,35.0,32.0,41.0,18.0,13.0
1,81.0,30.0,15.0,4.0,4.0,4.0,12.0,12.0,23.0,37.0,...,120.0,104.0,65.0,39.0,35.0,32.0,41.0,18.0,13.0,2.0
2,30.0,15.0,4.0,4.0,4.0,12.0,12.0,23.0,37.0,41.0,...,104.0,65.0,39.0,35.0,32.0,41.0,18.0,13.0,2.0,0.0
3,15.0,4.0,4.0,4.0,12.0,12.0,23.0,37.0,41.0,103.0,...,65.0,39.0,35.0,32.0,41.0,18.0,13.0,2.0,0.0,2.0
4,4.0,4.0,4.0,12.0,12.0,23.0,37.0,41.0,103.0,97.0,...,39.0,35.0,32.0,41.0,18.0,13.0,2.0,0.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714,99.0,74.0,50.0,33.0,16.0,17.0,3.0,1.0,0.0,1.0,...,67.0,73.0,97.0,106.0,107.0,109.0,96.0,107.0,156.0,108.0
715,74.0,50.0,33.0,16.0,17.0,3.0,1.0,0.0,1.0,1.0,...,73.0,97.0,106.0,107.0,109.0,96.0,107.0,156.0,108.0,88.0
716,50.0,33.0,16.0,17.0,3.0,1.0,0.0,1.0,1.0,2.0,...,97.0,106.0,107.0,109.0,96.0,107.0,156.0,108.0,88.0,81.0
717,33.0,16.0,17.0,3.0,1.0,0.0,1.0,1.0,2.0,16.0,...,106.0,107.0,109.0,96.0,107.0,156.0,108.0,88.0,81.0,49.0


In [17]:
targets_one_location = pd.DataFrame(y, columns=[f'target_rides_next_hour'])
targets_one_location

,target_rides_next_hour
0,2.0
1,0.0
2,2.0
3,2.0
4,0.0
...,...
714,88.0
715,81.0
716,49.0
717,44.0


In [22]:
from tqdm import tqdm

def transform_ts_data_into_features_and_target(
        ts_data: pd.DataFrame,
        input_seq_len: int,
        step_size: int
) -> pd.DataFrame:
    
    assert set(ts_data.columns) == {'pickup_hour', 'rides','pickup_location_id'}

    location_ids = ts_data['pickup_location_id'].unique()
    features = pd.DataFrame()
    targets = pd.DataFrame()

    for location_id in tqdm(location_ids):

        ts_data_one_location = ts_data.loc[
            ts_data.pickup_location_id == location_id,
            ['pickup_hour', 'rides']
        ]

        indices = get_cutoff_indices(
            ts_data_one_location,
            input_seq_len,
            step_size
        )

        n_examples = len(indices)
        x = np.ndarray(shape=(n_examples, input_seq_len), dtype=np.float32)
        y = np.ndarray(shape=(n_examples), dtype=np.float32)
        pickup_hours = []

        for i, idx in enumerate(indices):
            x[i, :] = ts_data_one_location.iloc[idx[0]:idx[1]]['rides'].values
            y[i] = ts_data_one_location.iloc[idx[1]:idx[2]]['rides'].values[0]
            pickup_hours.append(ts_data_one_location.iloc[idx[1]]['pickup_hour'])

        features_one_location = pd.DataFrame(
            x,
            columns=[f'rides_previous_{i+1}_hour' for i in reversed(range(input_seq_len))]
        )
        features_one_location['pickup_hour'] = pickup_hours
        features_one_location['pickup_location_id'] = location_id

        targets_one_location = pd.DataFrame(y, columns=[f'target_rides_next_hour'])

        features = pd.concat([features, features_one_location])
        targets = pd.concat([targets, targets_one_location])

    features.reset_index(inplace=True, drop=True)
    targets.reset_index(inplace=True, drop=True)

    return features, targets['target_rides_next_hour']

In [24]:
features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*7*1, #one week
    step_size=24
)

print(f'{features.shape=}')
print(f'{targets.shape=}')

100%|██████████| 257/257 [00:02<00:00, 96.57it/s]

features.shape=(6168, 170)
targets.shape=(6168,)
